In [ ]:
# Get source code (neural network and custom configs)
! git clone https://github.com/vromash/smoke-detection-camera.git configs
! git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'configs'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 1032 (delta 75), reused 107 (delta 40), pack-reused 867
Receiving objects: 100% (1032/1032), 428.35 MiB | 22.04 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (735/735), done.
Cloning into 'darknet'...
remote: Enumerating objects: 14665, done.
remote: Total 14665 (delta 0), reused 0 (delta 0), pack-reused 14665
Receiving objects: 100% (14665/14665), 13.25 MiB | 987.00 KiB/s, done.
Resolving deltas: 100% (9977/9977), done.


In [ ]:
# Build network
! cd darknet; ./build.sh

-- Autodetected CUDA architecture(s):  6.0
-- Building with CUDA flags: -gencode;arch=compute_60,code=sm_60
-- Your setup does not supports half precision (it requires CC >= 7.0)
-- Found OpenMP_C: -fopenmp  
-- Found OpenMP_CXX: -fopenmp  
-- Found OpenMP: TRUE   
--   ->  darknet is fine for now, but uselib_track has been disabled!
--   ->  Please rebuild OpenCV from sources with CUDA support to enable it
-- CMAKE_CUDA_FLAGS: -gencode arch=compute_60,code=sm_60 --compiler-options " -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -Wno-deprecated-declarations -Wno-write-strings -DGPU -DCUDNN -DOPENCV -fPIC -fopenmp -Ofast " 
-- Configuring done
-- Generating done
-- Build files have been written to: /content/darknet/build_release
[ 88%] Built target dark
[ 98%] Built target darknet
[100%] Built target uselib
Install the project...
-- Install configuration: "Release"
-- Up-to-date: /content/darknet/libdarknet.so
-- Up-to-date: /content/darknet/include/darknet/darknet.h
-- U

In [ ]:
# Copy configs into network
! cp -R configs/obj darknet/build/darknet/x64/data/
! cp configs/obj.data darknet/build/darknet/x64/data/
! cp configs/obj.names darknet/build/darknet/x64/data/
! cp configs/train.txt darknet/build/darknet/x64/data/
! cp configs/test.txt darknet/build/darknet/x64/data/
! cp configs/yolov4-tiny.conv.29 darknet/build/darknet/x64/
! cp configs/yolov4-tiny-obj.cfg darknet/cfg/
! cp configs/weights/1301/yolov4-tiny-obj_last.weights darknet/build/darknet/x64/backup/

In [ ]:
# Install video downloader
! wget https://yt-dl.org/downloads/latest/youtube-dl -O /usr/local/bin/youtube-dl
! chmod a+rx /usr/local/bin/youtube-dl

--2021-01-13 10:36:14--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2021.01.08/youtube-dl [following]
--2021-01-13 10:36:14--  https://yt-dl.org/downloads/2021.01.08/youtube-dl
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ytdl-org/youtube-dl/releases/download/2021.01.08/youtube-dl [following]
--2021-01-13 10:36:14--  https://github.com/ytdl-org/youtube-dl/releases/download/2021.01.08/youtube-dl
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-production-release-asset-2e65be.s3.amazonaws.com/1039520/7639

In [ ]:
# Download ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip

--2021-01-13 10:50:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 107.21.11.91, 52.23.32.39, 52.20.200.43, ...
Connecting to bin.equinox.io (bin.equinox.io)|107.21.11.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2021-01-13 10:50:08 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
! pip install -r configs/backend/requirements.txt

     |████████████████████████████████| 51kB 8.7MB/s 
  Created wheel for ushlex: filename=ushlex-0.99.1-cp36-none-any.whl size=4417 sha256=e3b4d8867452295fa9e251d075a87e26351cae693a06e14dc72018d71bfd2768
  Stored in directory: /root/.cache/pip/wheels/b8/f2/ed/ea55f33eb524ba8afcfd55e93a6facc96ff23d637e4e609f6d
Successfully built ushlex


In [ ]:
from IPython import get_ipython

In [ ]:
# Start application in background
! fuser -k 5000/tcp
get_ipython().system_raw('python3 configs/backend/app/app.py &')

5000/tcp:             2173  2176


In [ ]:
# Start ngrok
get_ipython().system_raw('./ngrok http 5000 &')

In [ ]:
# Get ngrok ip
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://c9d44dc7f7d9.ngrok.io
